In [ ]:
import os, sys

current_path = '/content/drive/My Drive/Colab Notebooks/Project_Stock/'

os.chdir(current_path)

In [ ]:
# pickle.format_version
# print(pd.show_versions())
!pip install pandas==0.25.3
# !pip install mpl_finance

### **ARIMA**

In [ ]:
from statsmodels.tsa.arima_model import ARIMA
from datetime import datetime


def arima_test(close, use_rows=None):

  size = int(len(close) * 0.66)
  train, test = close[0:size].values, close[size:len(close)]
  test_shift = test.shift(1).values
  test = test.values
  # break

  history = list(train)
  predictions = list()
  err_ranges = list()
  for t in range(len(test)):
    
      if use_rows is not None:
        history = history[-use_rows:]
        
      model = ARIMA(history, order=(0, 2, 1))
      model_fit = model.fit(trend='c', disp=0)
      output = model_fit.forecast()
      # print(output)
      # break
      yhat = output[0]
      predictions.append(yhat)
      err_ranges.append(output[1])
      obs = test[t]
      # print('obs :', obs)
      history.append(obs)
      # break
      print('\r %.2f%%' % (t / len(test) * 100), end='')

  print(len(test), len(predictions))

  return predictions, err_ranges


# print(high)


def get_back_result(ohlcv, predictions, err_ranges, tp=0.04, sl=None, leverage=1, show_detail=False, show_plot=False, return_pr=False, cumsum=False, 
                    close_ver=False, reverse_short=False):

  
  high, low, test = np.split(ohlcv.values[-len(predictions):, [1, 2, 3]], 3, axis=1)

  if close_ver:
    predictions = ohlcv['close'].shift(1).values[-len(test):]

  fee = 0.0006
  long_profits = []
  short_profits = []
  liquidations = []
  win_cnt = 0
  for i in range(len(test)):

    long_ep = (predictions[i] - err_ranges[i]) * (1 / (tp + 1))
    if sl is not None:
      long_sl = long_ep * (1 / (sl + 1))

    # assert long_ep < long_exit, 'long_exit < long_ep !, %s, %s' % (long_exit, long_ep)
    
    short_ep = (predictions[i] + err_ranges[i]) * (1 + tp)
    # short_ep = (predictions[i] + err_ranges[i]) * (1 / (1 - tp))
    if sl is not None:
      short_sl = short_ep * (1 / (1 - sl))

    # print((low[i]))

    #    long 우선   # <-- long & short 둘다 체결된 상황에서는 long 체결을 우선으로 한다.
    if low[i] < long_ep:
      
      liquidation = low[i] / long_ep - fee
      l_liquidation = 1 + (liquidation - 1) * leverage
      liquidations.append(l_liquidation)

      if max(l_liquidation, 0) == 0:
        l_profit = 0
        # print('low[i], long_ep, l_liquidation :', low[i], long_ep, l_liquidation)
      else:

        if sl is not None:
          if low[i] < long_sl:
            profit = long_sl / long_ep - fee
          else:
            profit = test[i] / long_ep - fee

        else:
          profit = test[i] / long_ep - fee

        l_profit = 1 + (profit - 1) * leverage
        l_profit = max(l_profit, 0)
        
        if profit >= 1:
          win_cnt += 1

      long_profits.append(l_profit)
      short_profits.append(1.0)

      if show_detail:
        print(test[i], predictions[i], long_ep)

    # if high[i] > short_ep > low[i]: # 지정 대기가 아니라, 해당 price 가 지나면, long 한다.

    #   if not reverse_short:
    #     liquidation = short_ep / high[i]  - fee
    #   else:
    #     liquidation = low[i] / short_ep  - fee
    #   l_liquidation = 1 + (liquidation - 1) * leverage

    #   if max(l_liquidation, 0) == 0:
    #     l_profit = 0
    #   else:

    #     if sl is not None:
    #       if high[i] > short_sl:

    #         if not reverse_short:
    #           profit = short_ep / short_sl - fee
    #         else:
    #           profit = short_sl / short_ep - fee

    #       else:
    #         if not reverse_short:
    #           profit = short_ep / test[i] - fee
    #         else:
    #           profit = test[i] / short_ep - fee

    #     else:

    #       if not reverse_short:
    #         profit = short_ep / test[i] - fee
    #       else:
    #         profit = test[i] / short_ep - fee

    #     l_profit = 1 + (profit - 1) * leverage
    #     l_profit = max(l_profit, 0)

    #     if profit >= 1:
    #       win_cnt += 1

    #   short_profits.append(l_profit)
    #   long_profits.append(1.0)

    #   if show_detail:
    #     print(test[i], predictions[i], short_ep)
    
    else:
      long_profits.append(1.0)
      short_profits.append(1.0)
      liquidations.append(1.0)


  long_win_ratio = sum(np.array(long_profits) > 1.0) / sum(np.array(long_profits) != 1.0)
  short_win_ratio = sum(np.array(short_profits) > 1.0) / sum(np.array(short_profits) != 1.0)
  long_frequency = sum(np.array(long_profits) != 1.0) / len(test)
  short_frequency = sum(np.array(short_profits) != 1.0) / len(test)
  if not cumsum:
    long_accum_profit = np.array(long_profits).cumprod()
    short_accum_profit = np.array(short_profits).cumprod()
  else:
    long_accum_profit = (np.array(long_profits) - 1.0).cumsum()
    short_accum_profit = (np.array(short_profits) - 1.0).cumsum()

  # print(win_ratio)

  if show_plot:

    plt.figure(figsize=(10, 5))
    plt.suptitle('tp=%.4f, lvrg=%d' % (tp, leverage))

    plt.subplot(151)
    plt.plot(liquidations)
    plt.title('liquidations')

    plt.subplot(152)
    plt.plot(long_profits)
    plt.title('Win Ratio : %.2f %%\nrequency : %.2f %%' % (long_win_ratio * 100, long_frequency * 100), color='black')
    # plt.show()

    # print()
    plt.subplot(153)
    plt.plot(long_accum_profit)
    plt.title('Accum_profit : %.2f' % long_accum_profit[-1], color='black')

    plt.subplot(154)
    plt.plot(short_profits)
    plt.title('Win Ratio : %.2f %%\nrequency : %.2f %%' % (short_win_ratio * 100, short_frequency * 100), color='black')
    # plt.show()

    # print()
    plt.subplot(155)
    plt.plot(short_accum_profit)
    plt.title('Accum_profit : %.2f' % short_accum_profit[-1], color='black')
    plt.show()

  return [long_win_ratio, short_win_ratio], [long_frequency, short_frequency], [long_accum_profit[-1], short_accum_profit[-1]], [long_profits, short_profits]


# get_back_result(tp=0.04, leverage=1, show_plot=True)


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

interval = '30m'
date_path = './candlestick_concated/%s/' % interval
file_list = os.listdir(date_path)

print((file_list))

### **Add coin**

In [ ]:
import pickle

with open('./arima_result/arima_profit_result_%s.pickle' % interval, 'rb') as f:
  load_candi_pr_dict = pickle.load(f)

result_dict = dict()

for file in file_list:

  # file = '2021-02-07 ETH.xlsx'
  print(file)
  if not file.endswith('xlsx'):
    continue

  #     add new data    #
  elif file not in list(load_candi_pr_dict.keys()):

    ohlcv = pd.read_excel(date_path + file, index_col=0)
    print(len(ohlcv))
    # print(ohlcv.head())

    temp_dict =  dict()
    temp_dict['ohlcv'] = ohlcv
    # print(temp_dict)
    load_candi_pr_dict[file] = temp_dict

    # break

    close = ohlcv.iloc[:, [3]]
    predictions, err_ranges = arima_test(close, 3000)

    temp_dict['predictions'] = predictions
    temp_dict['err_ranges'] = err_ranges
    # result_dict[file] = temp_dict
    load_candi_pr_dict[file] = temp_dict

  # break

#         save dict       #
# with open('./arima_result/arima_test_result_%s.pickle' % interval, 'wb') as f:
  # pickle.dump(load_candi_pr_dict, f)

### **Load & Individually Plotting & Add Backtest_Result**

In [ ]:
import pickle
import time
import datetime

with open('./arima_result/arima_test_result_%s.pickle' % interval, 'rb') as f:
  load_dict = pickle.load(f)

tp_list = np.arange(-0.03, 0.05, 0.001)
leverage_list = np.arange(1, 6, 1)
safety_threshold = 0.
long_index = 0
leverage = 3

def get_tp(ohlcv, predictions, err_ranges):

    wr_list, fr_list, ap_list = list(), list(), list()
    max_ap_list = list()  

    progress_cnt = 0
    # selected_ap_list = list()
    start_time = time.time()

    for tp in tp_list:

      progress_cnt += 1

      if progress_cnt != 1:

        if progress_cnt == 2:
          a_loop_time = time.time() - start_time

        r_time = a_loop_time * (len(tp_list) - progress_cnt)
        # print(r_time)
        print('\r remaining time : %s' % (datetime.timedelta(seconds=r_time)), end='')
      # print('\r %.2f%%' % (progress_cnt / len(tp_list) * 100), end='')

      #     leverage analysis     #
      temp_ap_list = list()
      for leverage in leverage_list:

        try:
          result = get_back_result(ohlcv, predictions, err_ranges, tp=tp, leverage=leverage, reverse_short=False)

          # print(np.min(result[-1]))
          temp_ap_list.append(result[2])
          # print('leverage, result[2] :', leverage, result[2])

          if round(leverage) == 1:

            wr_list.append(result[0])
            fr_list.append(result[1])
            ap_list.append(result[2])

            break # <-- 어차피 acc 와 max acc 를 결정하는 tp 는 큰 차이가 없다고 생각해, 
            #  소모 시간을 줄이기 위해 break

          # if leverage > 1:
          #   break

        except Exception as e:
          print(e)
          break

      # print('np.max(temp_ap_list, axis=0) :', np.max(temp_ap_list, axis=0))
      max_ap_list.append(np.max(temp_ap_list, axis=0))

    # print(wr_list)
    # break

    #     stack dict    #
    temp_dict = load_dict[key]
    temp_dict['tp_list'] = tp_list[:len(ap_list)]
    wr_list = np.array(wr_list).reshape(-1, 2)
    fr_list = np.array(fr_list).reshape(-1, 2)
    ap_list = np.array(ap_list).reshape(-1, 2)
    max_ap_list = np.array(max_ap_list).reshape(-1, 2)

    temp_dict['wr_list'] = wr_list
    temp_dict['fr_list'] = fr_list
    temp_dict['ap_list'] = ap_list
    temp_dict['max_ap_list'] = max_ap_list


    #     individual plot   #
    plt.figure(figsize=(15, 5))
    plt.suptitle('get best tp & figure')

    plt.subplot(151)
    plt.plot(tp_list[:len(ap_list)], wr_list)
    plt.title('win ratio')

    plt.subplot(152)
    plt.plot(tp_list[:len(ap_list)], fr_list)
    plt.title('frequency')

    plt.subplot(153)
    plt.plot(tp_list[:len(ap_list)], ap_list)
    argmax_l, argmax_s = np.argmax(ap_list, axis=0)   # --> best tp in basic_acc
    plt.axvline(tp_list[:len(ap_list)][argmax_l], linestyle='--')
    plt.axvline(tp_list[:len(ap_list)][argmax_s], linestyle='--')
    plt.title('acc profit, max at l:%.4f, s:%.4f' % (tp_list[:len(ap_list)][argmax_l], tp_list[:len(ap_list)][argmax_s]))  

    return_tp = tp_list[:len(ap_list)][argmax_l]

    plt.subplot(154)
    plt.plot(tp_list[:len(max_ap_list)], max_ap_list[:, [long_index]]) 
    argmax_l, argmax_s = np.argmax(max_ap_list, axis=0)  # --> best tp in basic_acc
    plt.axvline(tp_list[:len(max_ap_list)][argmax_l], linestyle='--')
    plt.axvline(tp_list[:len(max_ap_list)][argmax_s], linestyle='--')
    plt.title('max acc profit by leverage\n max at l:%.4f, s:%.4f' % (tp_list[:len(max_ap_list)][argmax_l], tp_list[:len(max_ap_list)][argmax_s])) 

    # plt.show()
    l_selected_tp = tp_list[:len(max_ap_list)][argmax_l]
    s_selected_tp = tp_list[:len(max_ap_list)][argmax_s]
    #     leverage analysis     #
    l_selected_ap_list = list()
    s_selected_ap_list = list()
    for leverage in leverage_list:

      try:
        l_result = get_back_result(ohlcv, predictions, err_ranges, tp=l_selected_tp, leverage=leverage, reverse_short=False)
        s_result = get_back_result(ohlcv, predictions, err_ranges, tp=s_selected_tp, leverage=leverage, reverse_short=False)
        
        if min(s_result[-1][long_index]) < safety_threshold:
          continue

        l_selected_ap_list.append(l_result[2])
        s_selected_ap_list.append(s_result[2])

      except Exception as e:
        print(e)
        break
    
    #     stack dict    #
    temp_dict['leverage_list'] = leverage_list[:len(s_selected_ap_list)]
    l_selected_ap_list = np.array(l_selected_ap_list).reshape(-1, 2)[:, :1]
    s_selected_ap_list = np.array(s_selected_ap_list).reshape(-1, 2)[:, 1:]
    temp_dict['leverage_ap_list'] = np.hstack((l_selected_ap_list, s_selected_ap_list))
    
    plt.subplot(155)
    plt.plot(leverage_list[:len(l_selected_ap_list)], l_selected_ap_list)
    plt.plot(leverage_list[:len(s_selected_ap_list)], s_selected_ap_list)
    l_argmax = np.argmax(l_selected_ap_list)
    s_argmax = np.argmax(s_selected_ap_list)
    plt.title('acc profit by leverage\n max at l:tp=%.3f lvrg=%.0f\n s:tp=%.3f lvrg=%.0f' \
              % (l_selected_tp, leverage_list[:len(l_selected_ap_list)][l_argmax], s_selected_tp, leverage_list[:len(s_selected_ap_list)][s_argmax]))  
    plt.axvline(leverage_list[:len(l_selected_ap_list)][l_argmax], linestyle='--')
    plt.axvline(leverage_list[:len(s_selected_ap_list)][s_argmax], linestyle='--')
    plt.xlim(0, 13)

    # plt.show()
    plt.close()

    return return_tp

# tp_list = [selected_tp]

for file in file_list:

  # file = '2021-02-07 ETH.xlsx'
  print(file)
  if not file.endswith('xlsx'):
    continue

  key = file

  ohlcv = pd.read_excel(date_path + file, index_col=0)
  print(len(ohlcv)) 


# for key in load_dict.keys():

  # print(key)

  # if len(list(load_dict[key].keys())) > 3:
  #   print(list(load_dict[key].keys()))
  #   continue

  #       tp analysis     #
  # ohlcv = load_dict[key]['ohlcv']
  predictions = load_dict[key]['predictions']
  err_ranges = load_dict[key]['err_ranges']
  # predictions = ohlcv['close'].shift(1).values
  # err_ranges = np.zeros_like(predictions)

  global_tp = get_tp(ohlcv, predictions, err_ranges)

  #   slice usable ohlcv  #
  ohlcv = ohlcv.iloc[-len(predictions):]

  split_size = 3
  a_size = int(len(ohlcv) / 10)

  global_acc_pr, local_acc_pr = [], []

  #     split data    #
  for i in range(split_size):

    if i != split_size - 1:

        if i !=0 :
          prev_ohlcv = temp_ohlcv
          prev_predictions = temp_predictions
          prev_err_ranges = temp_err_ranges

        temp_ohlcv = ohlcv.iloc[a_size * i : a_size * (i + 1)]
        temp_predictions = predictions[a_size * i : a_size * (i + 1)]
        temp_err_ranges = err_ranges[a_size * i : a_size * (i + 1)]

    else:

      if i !=0 :
          prev_ohlcv = temp_ohlcv
          prev_predictions = temp_predictions
          prev_err_ranges = temp_err_ranges

      temp_ohlcv = ohlcv.iloc[a_size * i : ]
      temp_predictions = predictions[a_size * i : ]
      temp_err_ranges = err_ranges[a_size * i : ]

    if i != 0:      
    
      #     check inner data --> prev && curr   #
      print('prev_predictions[:5] :', prev_predictions[:5])
      print('temp_predictions[:5] :', temp_predictions[:5])

      #   get prev_best_tp    #
      prev_best_tp = get_tp(prev_ohlcv, prev_predictions, prev_err_ranges)

      result = get_back_result(temp_ohlcv, temp_predictions, temp_err_ranges, tp=prev_best_tp, leverage=leverage, show_plot=False)
      local_acc_pr.append(result[2][0])
      result = get_back_result(temp_ohlcv, temp_predictions, temp_err_ranges, tp=global_tp, leverage=leverage, show_plot=False)
      global_acc_pr.append(result[2][0])

  xticks = ['local', 'global']
  yticks = [np.cumprod(local_acc_pr)[-1], np.cumprod(global_acc_pr)[-1]]
  # print('yticks :', yticks)
  plt.bar(xticks, yticks)
  # plt.bar())
  plt.show()
  # print()
  # break   # temporary adujst on one pair

  


#         save dict 2       #
# with open('./arima_result/arima_profit_ls_only_long_result_%s.pickle' % interval, 'wb') as f:
#   pickle.dump(load_dict, f)
  

### **Keep Connection**

In [ ]:
while 1: 1 

### **Sum Plot**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
# print(type(result_dict))

with open('./arima_result/arima_profit_ls_only_long_result_%s.pickle' % interval, 'rb') as f:
# with open('./arima_result/arima_profit_ls_only_long_expanded_result_%s.pickle' % interval, 'rb') as f:
  profit_result_dict = pickle.load(f)

keys = profit_result_dict.keys()
print(keys)
# keys = ['2021-02-07 ETH.xlsx']
# break


for i in range(2):

  plt.figure(figsize=(30, 5))

  plt.subplot(151)
  plt.title('win ratio')  
  for key in keys:  
    # print(profit_result_dict[key]['wr_list'].shape)
    plt.plot(profit_result_dict[key]['tp_list'], profit_result_dict[key]['wr_list'][:, [i]])


  plt.subplot(152)
  plt.title('frequency')  
  for key in keys:  
    plt.plot(profit_result_dict[key]['tp_list'], profit_result_dict[key]['fr_list'][:, [i]])

    
  plt.subplot(153)
  plt.title('acc profit')  
  for key in keys:  
    plt.plot(profit_result_dict[key]['tp_list'], profit_result_dict[key]['ap_list'][:, [i]])

  plt.subplot(155)
  plt.title('max acc profit by leverage')  
  for key in keys:  
    plt.plot(profit_result_dict[key]['tp_list'], profit_result_dict[key]['max_ap_list'][:, [i]], label=key)
    
  plt.subplot(154)
  plt.title('acc profit by leverage')  
  for key in keys:  
    plt.plot(profit_result_dict[key]['leverage_list'], profit_result_dict[key]['leverage_ap_list'][:, [i]], label=key)
    # print(result_dict[key]['leverage_ap_list'])
  plt.legend()

  plt.show()


### **Extract Candidate pairs**

In [ ]:

pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 2500)
pd.set_option('display.max_columns', 2500)

pairs = list(profit_result_dict.keys())
result_df = pd.DataFrame(index=pairs)
profit = list()
max_lv_profit = list()
pr_fr_std = list()
max_pr_fr_std = list()

long_index = 0

for key in keys:  
  # print(np.max(profit_result_dict[key]['ap_list']))
  profit.append(np.max(profit_result_dict[key]['ap_list'][:, [long_index]]))
  # print(np.max(profit_result_dict[key]['leverage_ap_list']))
  max_lv_profit.append(np.max(profit_result_dict[key]['leverage_ap_list'][:, [long_index]]))

  ohlcv = profit_result_dict[key]['ohlcv']
  predictions = profit_result_dict[key]['predictions']
  err_ranges = profit_result_dict[key]['err_ranges']
  
  #             편의를 위해 long position 분석만을 진행한다.           #
  argmax = np.argmax(profit_result_dict[key]['ap_list'][:, [long_index]])
  peak_tp = profit_result_dict[key]['tp_list'][argmax]

  result = get_back_result(ohlcv, predictions, err_ranges, tp=peak_tp, leverage=4)
  profit_list = result[3][long_index]
  pr_list_fr = list(map(lambda x : 0 if x == 1.0 else 1, profit_list))
  profited_index = np.argwhere(np.array(pr_list_fr) == 1)
  # print(profited_index)
  term_list = list()
  for i in range(len(profited_index) - 1):
    term_list.append(profited_index[i + 1] - profited_index[i])

  profit_result_dict[key]['pr_list'] = profit_list
  profit_result_dict[key]['pr_fr_std'] = np.std(term_list)
  pr_fr_std.append(np.std(term_list))

  argmax = np.argmax(profit_result_dict[key]['max_ap_list'][:, [long_index]])
  max_peak_tp = profit_result_dict[key]['tp_list'][argmax]

  result = get_back_result(ohlcv, predictions, err_ranges, tp=max_peak_tp, leverage=4)
  profit_list = result[3][long_index]
  pr_list_fr = list(map(lambda x : 0 if x == 1.0 else 1, profit_list))
  profited_index = np.argwhere(np.array(pr_list_fr) == 1)
  # print(profited_index)
  term_list = list()
  for i in range(len(profited_index) - 1):
    term_list.append(profited_index[i + 1] - profited_index[i])

  profit_result_dict[key]['max_pr_list'] = profit_list
  profit_result_dict[key]['max_pr_fr_std'] = np.std(term_list)
  max_pr_fr_std.append(np.std(term_list))


# result_df['index'] = pairs
result_df['profit']= profit
result_df['max_lv_profit']= max_lv_profit
result_df['pr_fr_std'] = pr_fr_std
result_df['max_pr_fr_std'] = max_pr_fr_std
# print(result_df)

#     sort by values    #
print(result_df.sort_values(by='profit', ascending=False))
print(result_df.sort_values(by=['max_lv_profit'], ascending=False))
print(result_df.sort_values(by=['pr_fr_std'], ascending=True))
print(result_df.sort_values(by=['max_pr_fr_std'], ascending=True))

#     extract candidates    #
candis = result_df.sort_values(by=['max_lv_profit'], ascending=False).index
print(candis)



#         save dict 2       #
# with open('./arima_result/arima_candi_profit_ls_only_long_result_%s.pickle' % interval, 'wb') as f:
#   pickle.dump(profit_result_dict, f)

### **Candi Result Analysis**

In [ ]:
long_index = 0
safety_threshold = 0.6



for i in range(len(candis)):

  keys = [candis[i]]

  plt.figure(figsize=(35, 7))
  plt.suptitle('%s %s' % (interval, keys))

  plt.subplot(1,10,1)
  plt.title('win ratio')  
  for key in keys:  
    plt.plot(profit_result_dict[key]['tp_list'], profit_result_dict[key]['wr_list'][:, [long_index]])


  plt.subplot(1,10,2)
  plt.title('frequency')  
  for key in keys:  
    plt.plot(profit_result_dict[key]['tp_list'], profit_result_dict[key]['fr_list'][:, [long_index]])

    
  plt.subplot(1,10,3)
  for key in keys:  
    plt.plot(profit_result_dict[key]['tp_list'], profit_result_dict[key]['ap_list'][:, [long_index]])
    argmax = np.argmax(profit_result_dict[key]['ap_list'][:, [long_index]])
    peak_tp = profit_result_dict[key]['tp_list'][argmax]
    plt.axvline(peak_tp, linestyle='--')
    plt.title('acc profit, max at %.4f' % (peak_tp))  

  plt.subplot(1,10,4)
  plt.title('max acc profit by leverage')  
  for key in keys:  
    plt.plot(profit_result_dict[key]['tp_list'], profit_result_dict[key]['max_ap_list'][:, [long_index]], label=key)
    argmax = np.argmax(profit_result_dict[key]['max_ap_list'][:, [long_index]])
    max_peak_tp = profit_result_dict[key]['tp_list'][argmax]
    plt.axvline(max_peak_tp, linestyle='--')
    plt.title('max acc profit, max at %.4f' % (max_peak_tp))  


  for key in keys:  
    # print(profit_result_dict[key]['leverage_ap_list'])

    for tp in [peak_tp, max_peak_tp]:

      if tp == peak_tp:
        plt.subplot(1,10,5)
      else:
        plt.subplot(1,10,6)

      #     leverage analysis     #
      ohlcv = profit_result_dict[key]['ohlcv']
      predictions = profit_result_dict[key]['predictions']
      err_ranges = profit_result_dict[key]['err_ranges']
      leverage_list = profit_result_dict[key]['leverage_list']
      temp_ap_list = list()
      temp_pr_list = list()

      for leverage in leverage_list:

        try:
          result = get_back_result(ohlcv, predictions, err_ranges, tp=tp, leverage=leverage, reverse_short=False)

          # if min(result[-1][long_index]) < safety_threshold:
          #   continue

          temp_ap_list.append(result[2][long_index])
          temp_pr_list.append(result[3][long_index])

          # if round(leverage) == 1:
          #   temp_pr_list = result[3]

        except Exception as e:
          print(e)
          break

      
      # profit_result_dict[key]['pr_list'] = temp_pr_list

      plt.plot(profit_result_dict[key]['leverage_list'][:len(temp_ap_list)],temp_ap_list, label=key)
      argmax = np.argmax(temp_ap_list)
      plt.title('acc profit by leverage\n max at tp=%.4f lvrg=%.0f' % (tp, profit_result_dict[key]['leverage_list'][:len(temp_ap_list)][argmax]))  
      plt.axvline(profit_result_dict[key]['leverage_list'][:len(temp_ap_list)][argmax], linestyle='--')

      if tp == peak_tp:
        plt.subplot(1,10,7)
      else:
        plt.subplot(1,10,9)

      plt.plot(temp_pr_list[argmax])
    # plt.xlim(0, 13)

    
  # plt.subplot(1,10,7)
  for key in keys:  
    pr_list = profit_result_dict[key]['pr_list']
    # plt.plot(pr_list)

    pr_fr_std = profit_result_dict[key]['pr_fr_std']
    pr_list_fr = list(map(lambda x : 0 if x == 1.0 else 1, pr_list))

    plt.subplot(1,10,8)
    plt.title('std : %s' % pr_fr_std)
    plt.plot(pr_list_fr)

  # plt.subplot(1,10,9)
  for key in keys:  
    pr_list = profit_result_dict[key]['max_pr_list']
    # plt.plot(pr_list)

    pr_fr_std = profit_result_dict[key]['max_pr_fr_std']
    pr_list_fr = list(map(lambda x : 0 if x == 1.0 else 1, pr_list))

    plt.subplot(1,10,10)
    plt.title('std : %s' % pr_fr_std)
    plt.plot(pr_list_fr)


  
  

  plt.show()

  

#### **RM - Maximum Loss**

In [ ]:
leverage_list2 = np.arange(1, 6, 1)

for i in range(len(candis)):

  keys = [candis[i]]
  
  if 'eth'.upper() not in candis[i]:
    continue

  # plt.figure(figsize=(35, 10))
  # plt.suptitle('%s %s' % (interval, keys))

  # plt.subplot(1,10,3)
  for key in keys:  
    # plt.plot(profit_result_dict[key]['tp_list'], profit_result_dict[key]['ap_list'])
    argmax = np.argmax(profit_result_dict[key]['ap_list'][:, [long_index]])
    peak_tp = profit_result_dict[key]['tp_list'][argmax]
    # plt.axvline(peak_tp, linestyle='--')
    # plt.title('acc profit, max at %.4f' % (peak_tp))  

  # plt.subplot(1,10,4)
  # plt.title('max acc profit by leverage')  
  for key in keys:  
    # plt.plot(profit_result_dict[key]['tp_list'], profit_result_dict[key]['max_ap_list'], label=key)
    argmax = np.argmax(profit_result_dict[key]['max_ap_list'][:, [long_index]])
    max_peak_tp = profit_result_dict[key]['tp_list'][argmax]
    # plt.axvline(max_peak_tp, linestyle='--')
    # plt.title('max acc profit, max at %.4f' % (max_peak_tp))  


  for key in keys:  
    # print(profit_result_dict[key]['leverage_ap_list'])

    for tp in [peak_tp, max_peak_tp]:

      # if tp == peak_tp:
      #   plt.subplot(1,10,5)
      # else:
      #   plt.subplot(1,10,6)

      #     leverage analysis     #
      ohlcv = profit_result_dict[key]['ohlcv']
      predictions = profit_result_dict[key]['predictions']
      err_ranges = profit_result_dict[key]['err_ranges']
      # leverage_list = profit_result_dict[key]['leverage_list']
      # temp_ap_list = list()
      # temp_pr_list = list()

      for leverage in leverage_list2:

        try:
          print('-------------- %s --------------' % key)
          result = get_back_result(ohlcv, predictions, err_ranges, tp=tp, leverage=leverage, show_plot=True, reverse_short=False, show_detail=False)
          # temp_ap_list.append(result[2])
          # temp_pr_list.append(result[3])

          # if round(leverage) == 1:
          #   temp_pr_list = result[3]

        except Exception as e:
          print(e)
          break
        
  # break

  

### **Optimization : Classify**

In [ ]:

      
# from sklearn.cluster import KMeans
# from sklearn.metrics import silhouette_score
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import precision_score, recall_score, precision_recall_curve
# from sklearn.metrics import confusion_matrix
# from sklearn.metrics import plot_confusion_matrix


# from sklearn.utils.testing import all_estimators
# from sklearn.metrics import accuracy_score, classification_report

# from functools import partial
# from sklearn.utils import class_weight

# np.random.seed(1)
# tf.random.set_seed(1)

# # coin = 'dot'
# leverage = 3


# for i in range(len(candis)):

#   keys = [candis[i]]
  
#   # if 'algo'.upper() not in candis[i]:
#   #   continue
#   if '2021-02-11 DOTUSDT.xlsx' not in candis[i]:
#     continue

#   # plt.figure(figsize=(35, 10))
#   # plt.suptitle('%s %s' % (interval, keys))


#   #         get tp parameter        #

#   # plt.subplot(1,10,3)
#   # for key in keys:  
#   #   # plt.plot(profit_result_dict[key]['tp_list'], profit_result_dict[key]['ap_list'])
#   #   argmax = np.argmax(profit_result_dict[key]['ap_list'][:, [long_index]])
#   #   peak_tp = profit_result_dict[key]['tp_list'][argmax]
#   #   # plt.axvline(peak_tp, linestyle='--')
#   #   # plt.title('acc profit, max at %.4f' % (peak_tp))  

#   # plt.subplot(1,10,4)
#   # plt.title('max acc profit by leverage')  
#   for key in keys:  
#     # plt.plot(profit_result_dict[key]['tp_list'], profit_result_dict[key]['max_ap_list'], label=key)
#     argmax = np.argmax(profit_result_dict[key]['max_ap_list'][:, [long_index]])
#     max_peak_tp = profit_result_dict[key]['tp_list'][argmax]
#     # plt.axvline(max_peak_tp, linestyle='--')
#     # plt.title('max acc profit, max at %.4f' % (max_peak_tp))  


#   for key in keys:  
#     # print(profit_result_dict[key]['leverage_ap_list'])

#     for tp in [max_peak_tp]:

#       # if tp == peak_tp:
#       #   plt.subplot(1,10,5)
#       # else:
#       #   plt.subplot(1,10,6)

#       #     leverage analysis     #
#       ohlcv = profit_result_dict[key]['ohlcv']
#       predictions = profit_result_dict[key]['predictions']
#       err_ranges = profit_result_dict[key]['err_ranges']

#       # predictions = ohlcv['close'].shift(1).values
#       # err_ranges = np.zeros_like(predictions)

#       leverage_list = profit_result_dict[key]['leverage_list']
#       # temp_ap_list = list()
#       # temp_pr_list = list()

#       try:
#         print('-------------- %s --------------' % key)
#         result = get_back_result(ohlcv, predictions, err_ranges, tp=tp, leverage=leverage, show_plot=True, reverse_short=False, show_detail=False)
#         # temp_ap_list.append(result[2])
#         # temp_pr_list.append(result[3])

#         # if round(leverage) == 1:
#         #   temp_pr_list = result[3]
#         pr_list = result[3][long_index]

#       except Exception as e:
#         print(e)
#         break    
#   # break

#       from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler

#       pd.set_option('display.max_rows', 500)
#       pd.set_option('display.max_columns', 500)
#       pd.set_option('display.width', 1000)

#       #         clustering zone           #

#       #       set data features : ohlc, v, ep
#       ohlc = ohlcv.iloc[-len(predictions):, :4]
#       vol = ohlcv.iloc[-len(predictions):, [4]]
#       long_ep = (np.array(predictions) - np.array(err_ranges)) * (1 / (tp + 1))
#       long_ep = long_ep.reshape(-1, 1)

#       ohlcv['u_wick'] = ohlcv['high'] / np.maximum(ohlcv['close'] , ohlcv['open'])
#       ohlcv['d_wick'] = np.minimum(ohlcv['close'] , ohlcv['open']) / ohlcv['low']
#       ohlcv['body'] = ohlcv['close'] / ohlcv['open']

#       candle = ohlcv.iloc[-len(predictions):, -3:]


#       print('len(ohlc) :', len(ohlc))
#       print('long_ep.shape :', long_ep.shape)
#       print('len(pr_list) :', len(pr_list))


#       #       set params    #
#       period = 5
#       data_x, data_pr, data_updown = [], [], []

#       for i in range(period, len(predictions)):

#         #   pr_list != 1 인 데이터만 사용한다
#         # if 1:
#         if pr_list[i] != 1:
          
#           #   prediction 을 제외한 이전 데이터를 사용해야한다
#           temp_ohlc = ohlc.iloc[i - period : i].values
#           temp_long_ep = long_ep[i - period : i]
#           temp_vol = vol.iloc[i - period : i].values
#           temp_candle = candle.iloc[i - period : i].values

#           # print(temp_ohlc.shape)
#           # print(temp_long_ep.shape)
#           # print(temp_vol.shape)
#           # print(temp_candle.shape)
#           # break

#           #   stacking  
#           # temp_data = np.hstack((temp_ohlc, temp_long_ep, temp_vol, temp_candle))
#           temp_data = np.hstack((temp_ohlc, temp_long_ep, temp_vol))
#           # temp_data = np.hstack((temp_ohlc, temp_long_ep))
#           # temp_data = temp_vol

#           #   scaler 설정

#           #   ohlc & ep -> max_abs
#           # max_abs = MaxAbsScaler()
#           # temp_data[:, :5] = max_abs.fit_transform(temp_data[:, :5])


#           min_max = MinMaxScaler()
#           temp_data[:, :5] = min_max.fit_transform(temp_data[:, :5])


#           #   vol -> min_max
#           min_max = MinMaxScaler()
#           temp_data[:, [5]] = min_max.fit_transform(temp_data[:, [5]])


#           #   candle -> max_abs    
#           # max_abs = MaxAbsScaler()
#           # temp_data[:, -3:] = max_abs.fit_transform(temp_data[:, -3:])

#           # min_max = MinMaxScaler()
#           # temp_data[:, -3:] = min_max.fit_transform(temp_data[:, -3:])

#           if np.isnan(np.sum(temp_data)):
#             continue

#           data_x.append(temp_data)
#           data_pr.append(pr_list[i])
#           data_updown.append(ohlc['close'].iloc[i] / ohlc['open'].iloc[i])


#       print('np.array(data_x).shape :', np.array(data_x).shape)
#       # print(data_x[0])


#       #       Reshape data for image deep - learning     #
#       _, row, col = np.array(data_x).shape

#       input_x = np.array(data_x).reshape(-1, row, col, 1).astype(np.float32)

#       #     1c to 3c    #
#       input_x = input_x * np.ones(3, dtype=np.float32)[None, None, None, :]

#       input_pr = np.array(data_pr).reshape(-1, 1).astype(np.float32)
#       input_ud = np.array(data_updown).reshape(-1, 1).astype(np.float32)
#       print('input_x.shape :', input_x.shape)
#       print('input_x.dtype :', input_x.dtype)
#       print('input_pr.shape :', input_pr.shape)
#       print('input_ud.shape :', input_ud.shape)

#       # break


#       #       check input     #
#       for i in range(len(input_x)):
#         plt.imshow(input_x[i])
#         plt.axis('off')
#         plt.show()
#         break


#       #         train / test split      #
#       x_train, x_test_, pr_train, pr_test_, ud_train, ud_test_ = train_test_split(input_x, input_pr, input_ud, test_size=0.4, shuffle=True, random_state=124)
#       x_test, x_val, pr_test, pr_val, ud_test, ud_val = train_test_split(x_test_, pr_test_, ud_test_, test_size=0.5, shuffle=True, random_state=124)


#       #         pr label   #
#       # y_train = np.where(pr_train > 1, 1, 0).astype(np.float32)
#       # y_test = np.where(pr_test > 1, 1, 0).astype(np.float32)
#       # y_val = np.where(pr_val > 1, 1, 0).astype(np.float32)

#       #         up label      #
#       y_train = np.where(ud_train > 1, 1, 0).astype(np.float32)
#       y_test = np.where(ud_test > 1, 1, 0).astype(np.float32)
#       y_val = np.where(ud_val > 1, 1, 0).astype(np.float32)

#       print('pr_train[:5] :', pr_train[:5])
#       print('ud_train[:5] :', ud_train[:5])
#       print('y_train[:5] :', y_train[:5])
#       print('y_train.dtype :', y_train.dtype)

#       print('x_train.shape :', x_train.shape)
#       print('x_test.shape :', x_test.shape)
#       print('x_val.shape :', x_val.shape)
#       print('y_train.shape :', y_train.shape)
#       print('y_test.shape :', y_test.shape)
#       print('y_val.shape :', y_val.shape)

#       print('np.unique(y_train, return_counts=True :', np.unique(y_train, return_counts=True))
#       print('np.unique(y_val, return_counts=True :', np.unique(y_val, return_counts=True))

#       label = y_train.reshape(-1, )
#       class_weights = class_weight.compute_class_weight('balanced', 
#                                                           classes=np.unique(label),
#                                                           y=label)
#       class_weights = dict(enumerate(class_weights))
#       print('class_weights :', class_weights)

#       print('np.isnan(np.sum(x_train)) :', np.isnan(np.sum(x_train)))
#       print('np.isnan(np.sum(y_train)) :', np.isnan(np.sum(y_train)))


#       train_set_ = tf.data.Dataset.from_tensor_slices((x_train, y_train))
#       test_set_ = tf.data.Dataset.from_tensor_slices((x_test, y_test))
#       val_set_ = tf.data.Dataset.from_tensor_slices((x_val, y_val))

#       resize_shape = [row * 2, col * 2]
#       # resize_shape = [224, 224]

#       def preprocess(image, label, flip=False):
#         if flip:
#             flipped_image = tf.image.random_flip_left_right(image)
#             resized_image = tf.image.resize(flipped_image, resize_shape)
#         else:
#             resized_image = tf.image.resize(image, resize_shape)
#         final_image = keras.applications.xception.preprocess_input(resized_image * 255)
#         return  final_image, label

#       # Batch Size
#       batch_size = 32
      
#       train_set = train_set_.shuffle(1000).repeat(2)
#       train_set = train_set.map(partial(preprocess, flip=False)).batch(batch_size).prefetch(1)
#       val_set = val_set_.map(preprocess).batch(batch_size).prefetch(1)
#       test_set = test_set_.map(preprocess).batch(batch_size).prefetch(1)

#       # print('test_set[0] :', test_set[0])
#       # print('len(train_set) :', len(train_set))
      
#       for set in train_set:
#         print(set[0].numpy()[0].shape)
#         plt.imshow(set[0].numpy()[0, :, :, 0])
#         break

#       break

#### **Data stacking**

In [ ]:
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler


leverage = 2
prev_x = None
for i in range(len(candis)):

  keys = [candis[i]]
  
  # if 'algo'.upper() not in candis[i]:
  #   continue
  if '2021-03-02 DOTUSDT.xlsx' in candis[i]:
    # print('')
    continue

  # plt.figure(figsize=(35, 10))
  # plt.suptitle('%s %s' % (interval, keys))


  #         get tp parameter        #

  # plt.subplot(1,10,3)
  # for key in keys:  
  #   # plt.plot(profit_result_dict[key]['tp_list'], profit_result_dict[key]['ap_list'])
  #   argmax = np.argmax(profit_result_dict[key]['ap_list'][:, [long_index]])
  #   peak_tp = profit_result_dict[key]['tp_list'][argmax]
  #   # plt.axvline(peak_tp, linestyle='--')
  #   # plt.title('acc profit, max at %.4f' % (peak_tp))  

  # plt.subplot(1,10,4)
  # plt.title('max acc profit by leverage')  
  for key in keys:  
    # plt.plot(profit_result_dict[key]['tp_list'], profit_result_dict[key]['max_ap_list'], label=key)
    argmax = np.argmax(profit_result_dict[key]['max_ap_list'][:, [long_index]])
    max_peak_tp = profit_result_dict[key]['tp_list'][argmax]
    # plt.axvline(max_peak_tp, linestyle='--')
    # plt.title('max acc profit, max at %.4f' % (max_peak_tp))  


  for key in keys:  
    # print(profit_result_dict[key]['leverage_ap_list'])

    for tp in [max_peak_tp]:

      # if tp == peak_tp:
      #   plt.subplot(1,10,5)
      # else:
      #   plt.subplot(1,10,6)

      #     leverage analysis     #
      ohlcv = profit_result_dict[key]['ohlcv']
      predictions = profit_result_dict[key]['predictions']
      err_ranges = profit_result_dict[key]['err_ranges']

      # predictions = ohlcv['close'].shift(1).values
      # err_ranges = np.zeros_like(predictions)

      leverage_list = profit_result_dict[key]['leverage_list']
      # temp_ap_list = list()
      # temp_pr_list = list()

      try:
        print('-------------- %s --------------' % key)
        result = get_back_result(ohlcv, predictions, err_ranges, tp=tp, leverage=leverage, show_plot=True, reverse_short=False, show_detail=False)
        # temp_ap_list.append(result[2])
        # temp_pr_list.append(result[3])

        # if round(leverage) == 1:
        #   temp_pr_list = result[3]
        pr_list = result[3][long_index]

      except Exception as e:
        print(e)
        break    
  # break


      pd.set_option('display.max_rows', 500)
      pd.set_option('display.max_columns', 500)
      pd.set_option('display.width', 1000)

      #         clustering zone           #

      #       set data features : ohlc, v, ep
      ohlc = ohlcv.iloc[-len(predictions):, :4]
      vol = ohlcv.iloc[-len(predictions):, [4]]
      long_ep = (np.array(predictions) - np.array(err_ranges)) * (1 / (tp + 1))
      long_ep = long_ep.reshape(-1, 1)

      ohlcv['u_wick'] = ohlcv['high'] / np.maximum(ohlcv['close'] , ohlcv['open'])
      ohlcv['d_wick'] = np.minimum(ohlcv['close'] , ohlcv['open']) / ohlcv['low']
      ohlcv['body'] = ohlcv['close'] / ohlcv['open']

      candle = ohlcv.iloc[-len(predictions):, -3:]


      print('len(ohlc) :', len(ohlc))
      print('long_ep.shape :', long_ep.shape)
      print('len(pr_list) :', len(pr_list))


      #       set params    #
      period = 5
      data_x, data_pr, data_updown = [], [], []
      key_i = i

      for i in range(period, len(predictions)):

        #   pr_list != 1 인 데이터만 사용한다
        # if 1:
        if pr_list[i] != 1:
          
          #   prediction 을 제외한 이전 데이터를 사용해야한다
          temp_ohlc = ohlc.iloc[i - period : i].values
          temp_long_ep = long_ep[i - period : i]
          temp_vol = vol.iloc[i - period : i].values
          temp_candle = candle.iloc[i - period : i].values

          # print(temp_ohlc.shape)
          # print(temp_long_ep.shape)
          # print(temp_vol.shape)
          # print(temp_candle.shape)
          # break

          #   stacking  
          # temp_data = np.hstack((temp_ohlc, temp_long_ep, temp_vol, temp_candle))
          temp_data = np.hstack((temp_ohlc, temp_long_ep, temp_vol))
          # temp_data = np.hstack((temp_ohlc, temp_long_ep))
          # temp_data = temp_vol

          #   scaler 설정

          #   ohlc & ep -> max_abs
          # max_abs = MaxAbsScaler()
          # temp_data[:, :5] = max_abs.fit_transform(temp_data[:, :5])


          min_max = MinMaxScaler()
          temp_data[:, :5] = min_max.fit_transform(temp_data[:, :5])


          #   vol -> min_max
          min_max = MinMaxScaler()
          temp_data[:, [5]] = min_max.fit_transform(temp_data[:, [5]])


          #   candle -> max_abs    
          # max_abs = MaxAbsScaler()
          # temp_data[:, -3:] = max_abs.fit_transform(temp_data[:, -3:])

          # min_max = MinMaxScaler()
          # temp_data[:, -3:] = min_max.fit_transform(temp_data[:, -3:])

          if np.isnan(np.sum(temp_data)):
            continue

          data_x.append(temp_data)
          data_pr.append(pr_list[i])
          data_updown.append(ohlc['close'].iloc[i] / ohlc['open'].iloc[i])


      print('np.array(data_x).shape :', np.array(data_x).shape)
      # print(data_x[0])


      #       Reshape data for image deep - learning     #
      _, row, col = np.array(data_x).shape

      input_x = np.array(data_x).reshape(-1, row, col, 1).astype(np.float32)

      #     1c to 3c    #
      input_x = input_x * np.ones(3, dtype=np.float32)[None, None, None, :]

      input_pr = np.array(data_pr).reshape(-1, 1).astype(np.float32)
      input_ud = np.array(data_updown).reshape(-1, 1).astype(np.float32)
      print('input_x.shape :', input_x.shape)
      print('input_x.dtype :', input_x.dtype)
      print('input_pr.shape :', input_pr.shape)
      print('input_ud.shape :', input_ud.shape)

      #     do stacking   #
      if prev_x is None:
        prev_x = input_x
        prev_pr = input_pr
        prev_ud = input_ud
      else:
        total_x = np.vstack((prev_x, input_x))
        total_pr = np.vstack((prev_pr, input_pr))
        total_ud = np.vstack((prev_ud, input_ud))

        prev_x = total_x
        prev_pr = total_pr
        prev_ud = total_ud

        print('total_x.shape :', total_x.shape)
        print('total_pr.shape :', total_pr.shape)
        print('total_ud.shape :', total_ud.shape)
        

In [ ]:
import keras
import tensorflow as tf

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, precision_recall_curve
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix


from sklearn.utils.testing import all_estimators
from sklearn.metrics import accuracy_score, classification_report

from functools import partial
from sklearn.utils import class_weight

seed = 4
random_state = 2
np.random.seed(seed)
tf.random.set_seed(seed)

#         train / test split      #
x_train, x_test_, pr_train, pr_test_, ud_train, ud_test_ = train_test_split(total_x, total_pr, total_ud, test_size=0.4, shuffle=True, random_state=random_state)
x_test, x_val, pr_test, pr_val, ud_test, ud_val = train_test_split(x_test_, pr_test_, ud_test_, test_size=0.5, shuffle=True, random_state=random_state)


#         pr label   #
# y_train = np.where(pr_train > 1, 1, 0).astype(np.float32)
# y_test = np.where(pr_test > 1, 1, 0).astype(np.float32)
# y_val = np.where(pr_val > 1, 1, 0).astype(np.float32)

#         up label      #
y_train = np.where(ud_train > 1, 1, 0).astype(np.float32)
y_test = np.where(ud_test > 1, 1, 0).astype(np.float32)
y_val = np.where(ud_val > 1, 1, 0).astype(np.float32)

print('pr_train[:5] :', pr_train[:5])
print('ud_train[:5] :', ud_train[:5])
print('y_train[:5] :', y_train[:5])
print('y_train.dtype :', y_train.dtype)

print('x_train.shape :', x_train.shape)
print('x_test.shape :', x_test.shape)
print('x_val.shape :', x_val.shape)
print('y_train.shape :', y_train.shape)
print('y_test.shape :', y_test.shape)
print('y_val.shape :', y_val.shape)

def class_ratio(in_list):

  return in_list / in_list[1]

print('np.unique(y_train, return_counts=True :', np.unique(y_train, return_counts=True), class_ratio(np.unique(y_train, return_counts=True)[1]))
print('np.unique(y_val, return_counts=True :', np.unique(y_val, return_counts=True), class_ratio(np.unique(y_val, return_counts=True)[1]))
print('np.unique(y_test, return_counts=True :', np.unique(y_test, return_counts=True), class_ratio(np.unique(y_test, return_counts=True)[1]))

label = y_train.reshape(-1, )
class_weights = class_weight.compute_class_weight('balanced', 
                                                    classes=np.unique(label),
                                                    y=label)
class_weights = dict(enumerate(class_weights))
print('class_weights :', class_weights)

sample_weight = np.ones(shape=(len(y_train),))
sample_weight[(y_train == 1).reshape(-1,)] = 1.5
print('sample_weight[:20] :', sample_weight[:20])


print('np.isnan(np.sum(x_train)) :', np.isnan(np.sum(x_train)))
print('np.isnan(np.sum(y_train)) :', np.isnan(np.sum(y_train)))


train_set_ = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_set_ = tf.data.Dataset.from_tensor_slices((x_test, y_test))
val_set_ = tf.data.Dataset.from_tensor_slices((x_val, y_val))

resize_shape = [row * 2, col * 2]
# resize_shape = [224, 224]

def preprocess(image, label, flip=False):
  if flip:
      flipped_image = tf.image.random_flip_left_right(image)
      resized_image = tf.image.resize(flipped_image, resize_shape)
  else:
      resized_image = tf.image.resize(image, resize_shape)
  final_image = keras.applications.xception.preprocess_input(resized_image * 255)
  return  final_image, label

# Batch Size
batch_size = 32

train_set = train_set_.shuffle(1000).repeat(2)
train_set = train_set.map(partial(preprocess, flip=False)).batch(batch_size).prefetch(1)
val_set = val_set_.map(preprocess).batch(batch_size).prefetch(1)
test_set = test_set_.map(preprocess).batch(batch_size).prefetch(1)

#### **DL**

In [ ]:
base_model = keras.applications.xception.Xception(weights="imagenet",
                                                  include_top=False)

In [ ]:
# print(base_model.summary())
# take the average of each feature map for dimensionality reduction 
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
# softmax class probability layer with the chosen 30 categories
output = keras.layers.Dense(2, activation="softmax")(avg)
# output = keras.layers.Dense(2, activation="softmax")(base_model.output)
# combining the base model and the customized top layers 

In [ ]:
# for layer in base_model.layers:
#     # print(layer.name)
#     layer.trainable = False

# # training the top layers
# optimizer = keras.optimizers.SGD(lr=0.2, momentum=0.9, decay=0.01)
# model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
#               metrics=["accuracy"])
# history = model.fit(train_set, epochs=1000, validation_data=val_set)
# score = model.evaluate(test_set)

In [ ]:
keras.backend.clear_session()

import gc

try:
  del model
  gc.collect()
except:
  pass

# unfreeze the weights of the pre trained layers
for layer in base_model.layers:
    layer.trainable = True
# training the whole layers
optimizer = keras.optimizers.SGD(learning_rate=0.01, momentum=0.9,
                                 nesterov=True, decay=0.001)
# optimizer = keras.optimizers.Adam(learning_rate=0.001,
#                                  decay=0.001)
model = keras.models.Model(inputs=base_model.input, outputs=output)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])
# model.compile(loss="categorical_crossentropy", optimizer=optimizer,
#               metrics=["accuracy"])
# save best performance on the validation set

ckpt_path = current_path + 'ckpt/'
board_path = current_path + 'graph/'

model_name = 'classifier_%s_min_re.h5' % period
# model_name = 'classifier_%s_min.h5' % period

checkpoint_cb = keras.callbacks.ModelCheckpoint(ckpt_path + model_name, 
                                                save_best_only=True, 
                                                monitor='val_loss',
                                                mode='auto')
# interrupt the training when there is no progress on the validation set for 10 epocs
early_stopping_cb = keras.callbacks.EarlyStopping(patience=50, restore_best_weights=True)
#view the learning curves during training using tensorboard
# tensorboard_cb = keras.callbacks.TensorBoard(board_path)
history = model.fit(train_set, epochs=1000, 
                    class_weight=class_weights,
                    # sample_weight=sample_weight,
                    validation_data=val_set, 
                    shuffle=False,
                    callbacks=[checkpoint_cb, early_stopping_cb])
                    # callbacks=[checkpoint_cb, early_stopping_cb, tensorboard_cb])

In [ ]:
# model_name = "classifier_5_min.h5"  # <-- specifying model name
model = keras.models.load_model(ckpt_path + model_name)

In [ ]:
# model.evaluate(val_set)

In [ ]:
test_result = model.predict(test_set)

print('test_result.shape :', test_result.shape)
print('pr_val.shape :', pr_val.shape)


y_score = test_result[:, [1]]
print('y_test[:5] :', y_test.reshape(-1,)[:5])
print('np.unique(y_test) :', np.unique(y_test, return_counts=True))
print('y_score[:5] :', y_score[:5])
print('np.unique(y_score) :', np.unique(y_score, return_counts=True))

print('y_test.shape :', y_test.shape)
print('y_score.shape :', y_score.shape)

#     precision recall curve   #
precision, recall, threshold = precision_recall_curve(y_test, y_score)
precision, recall = precision[:-1], recall[:-1]

plt.plot(threshold, precision, label='precision')
plt.plot(threshold, recall, label='recall')
plt.legend()
plt.title('precision recall')
plt.show()
# print(y_pred)

# thresh = 0.19
# threshold = [thresh]
print('threshold :', threshold)

acc_pr_bythr = []
for thresh in threshold:

  y_pred = np.where(y_score[:, -1] > thresh, 1, 0)
  print('y_pred.shape :', y_pred.shape)
  # print('y_pred :', y_pred)

  #     compare precision     #

  print('precision :', precision_score(y_test, y_pred))
  print('recall :', recall_score(y_test, y_pred))
  print()

  print('np.isnan(np.sum(x_test)) :', np.isnan(np.sum(x_test)))
  print('np.isnan(np.sum(y_test)) :', np.isnan(np.sum(y_test)))

  # plot_confusion_matrix(best_model, x_test, y_test, normalize=None)
  # plt.show()  
  print()

  #     check win-ratio improvement     #
  cmat = confusion_matrix(y_test, y_pred)
  # print(cmat)
  # print(np.sum(cmat, axis=1))

  test_size = len(y_test)
  test_pr_list = pr_test
  print('origin ac_pr :', np.cumprod(test_pr_list)[-1])

  org_wr = np.sum(cmat, axis=1)[-1] / sum(np.sum(cmat, axis=1))
  ml_wr = cmat[1][1] / np.sum(cmat, axis=0)[-1]
  print('win ratio improvement %.2f --> %.2f' % (org_wr, ml_wr))

  # print('pr_test.shape :', pr_test.shape)

  # print(y_pred)
  # print(test_pr_list)
  pred_pr_list = np.where(y_pred == 1, test_pr_list.reshape(-1, ), 1.0)
  # print('pred_pr_list.shape :', pred_pr_list.shape)

  if np.cumprod(test_pr_list)[-1] < np.cumprod(pred_pr_list)[-1]:
    print('accum_pr increased ! : %.3f --> %.3f' % (np.cumprod(test_pr_list)[-1], np.cumprod(pred_pr_list)[-1]))
    print('thresh :', thresh)
    
  # if len(threshold) == 1:
    plt.figure(figsize=(10, 5))
    plt.subplot(121)
    plt.plot(np.cumprod(test_pr_list))
    plt.title('%.3f' % (np.cumprod(test_pr_list)[-1]))
  # plt.show()

    plt.subplot(122)
    plt.plot(np.cumprod(pred_pr_list))
    plt.title('%.3f' % (np.cumprod(pred_pr_list)[-1]))
    plt.show()


  acc_pr_bythr.append(np.cumprod(pred_pr_list)[-1])

print('acc_pr_bythr :', acc_pr_bythr)

plt.figure(figsize=(10, 5))
plt.subplot(121)
plt.plot(threshold, precision, label='precision')
plt.plot(threshold, recall, label='recall')
plt.legend()
plt.title('precision recall')
# plt.show()
plt.subplot(122)
plt.plot(threshold, acc_pr_bythr)
plt.axhline(np.cumprod(test_pr_list)[-1], linestyle='--', color='r')
plt.show()

#### **clustering output**

In [ ]:
# x_train = test_result.flatten().reshape(-1, 1)
x_train = test_result
print(x_train[:10])
# x_train = test_result[:, [1]]
pr_train = pr_test

print('x_train.shape :', x_train.shape)
print('pr_train.shape :', pr_train.shape)

K = range(2, 10)
s_dist = []
sil = []
for k in K:
  # if cen_data.shape[0] < k:
  #   break

  km = KMeans(n_clusters=k)
  km = km.fit(x_train)

  labels = km.labels_
  # print('len(labels) :', len(labels))
  # print('labels[:10] :', labels[:10])
  sil.append(silhouette_score(x_train, labels, metric='euclidean'))

  # inertia = km.inertia_
  # s_dist.append(inertia)

best_k = K[np.argmax(np.array(sil))]

plt.figure(figsize=(10, 5))
plt.plot(K, sil)
plt.axvline(best_k, linestyle='--')
# plt.plot(K, s_dist)
plt.show()





#   with best_k, label 별 pr_list 확인
km = KMeans(n_clusters=best_k)
km = km.fit(x_train)

labels = km.labels_

print(km.score(x_train))
print(len(labels), len(pr_train))





#   label 별로 profit 을 저장, 승률을 확인한다
label_types = np.unique(labels, return_counts=False)

label_pr_dict = {}
#   init dict   #
for label in label_types:
  label_pr_dict[label] = []
print(label_pr_dict)
# break

for i, (label, pr) in enumerate(zip(labels, pr_train)):
  label_pr_dict[label].append(pr[0])

  
# for label in label_types:
print(label_pr_dict)





def win_ratio(list_x):

  win_cnt = np.sum(np.array(list_x) > 1)
  return win_cnt / len(list_x)


def acc_pr(list_x):

  return np.cumprod(np.array(list_x))[-1]


for key in label_pr_dict:
  
  print(key, ':', 'win_ratio : %.2f' % (win_ratio(label_pr_dict[key])), 'acc_pr : %.2f' % (acc_pr(label_pr_dict[key])))





#     predict test && test 의 라벨에 따른 win_ratio 확인
# test_labels = km.predict(x_test)
# # print(test_labels)

# label_pr_dict = {}
# #   init dict   #
# for label in label_types:
#   label_pr_dict[label] = []
# print(label_pr_dict)
# # break

# for i, (label, pr) in enumerate(zip(test_labels, pr_test)):
#   label_pr_dict[label].append(pr[0])

# for key in label_pr_dict:

#   print(key, ':', 'win_ratio : %.2f' % (win_ratio(label_pr_dict[key])), 'acc_pr : %.2f' % (acc_pr(label_pr_dict[key])))

